# Sample code for exercise 2-2
2017 Aug. Tsuyoshi Okubo  
2018 Dec. modified

In this code, you can simulate MPS decompositionof the ground state of spin model on 1d chain.  
$$\mathcal{H} = \sum_{i} J_z S_{i,z}S_{i+1,z} + J_{xy} (S_{i,x}S_{i+1,x} + S_{i,y}S_{i+1,y}) - h_x \sum_i S_{i,x} + D\sum_i S_{i,z}^2$$

In this exercise, the MPS is truncated so that the maximum bond dimension is chi_max. Thus the energy calculated from MPS is larger than that of ED, if chi_max is smaller than the m**(N/2)

You can change   

- N: # of sites
- m: size of spin  (2S = 2m + 1)  
- Jz: amplitude of SzSz interaction  
- Jxy: amplitude of SxSx + SySy interaction  
- hx : amplitude of external field alogn x direction  
- D : Single ion anisotropy  
- (periodic: In this exercize, we only consider open boundary)

- chi_max: maxmum bond dimension at truncation



In [1]:
import numpy as np
import scipy.linalg as linalg
import ED
import TEBD
from matplotlib import pyplot

In [2]:
N=10          ## Chain length 
m = 3         ## m = 2S + 1, e.g. m=3 for S=1 
Jz = 1.0      ## Jz for SzSz interaction
Jxy = 1.0     ## Jxy for SxSx + SySy interaction
hx = 0.0      ## external field along x direction
D = 0.0       ## single ion anisotropy
#periodic = False ## in this exersize , we only consider open boundary
chi_max = 10      ## maxmum bond dimension at truncation

In [3]:
print("2S = m - 1, N-site spin chain")
print("N = "+repr(N))
print("m = "+repr(m))
print("Hamiltonian parameters:")
print("Jz = "+repr(Jz))
print("Jxy = "+repr(Jxy))
print("hx = "+repr(hx))
print("D = "+repr(D))

2S = m - 1, N-site spin chain
N = 10
m = 3
Hamiltonian parameters:
Jz = 1.0
Jxy = 1.0
hx = 0.0
D = 0.0


In [4]:
## Obtain the smallest eigenvalue
eig_val,eig_vec = ED.Calc_GS(m,Jz, Jxy,hx,D,N,k=1)
print("Ground state energy per bond= " +repr(eig_val[0]/(N-1)))

Ground state energy per bond= -1.4327289035789932


In [5]:
## Make exact MPS (from "left")
Tn = []
lam = [np.ones((1,))]
lam_inv = 1.0/lam[0]
R_mat = eig_vec[:,0].reshape(m,m**(N-1))

chi_l=1
for i in range(N-1):
    U,s,VT = linalg.svd(R_mat,full_matrices=False)
    chi_r = s.size

    Tn.append(np.tensordot(np.diag(lam_inv),U.reshape(chi_l,m,chi_r),(1,0)).transpose(1,0,2))
    lam.append(s)
    lam_inv = 1.0/s
    R_mat = np.dot(np.diag(s),VT).reshape(chi_r*m,m**(N-i-2))
    chi_l = chi_r
Tn.append(VT.reshape(m,m,1).transpose(1,0,2))
lam.append(np.ones((1,)))

In [6]:
## Truncation 
for i in range(N-1):
    chi = min(chi_max,lam[i+1].shape[0])
    lam[i+1]=lam[i+1][:chi]
    Tn[i]=Tn[i][:,:,:chi]
    Tn[i+1]=Tn[i+1][:,:chi,:]

In [7]:
## Calculate Energy
Env_left=[]
Env_right=[]
for i in range(N):
    Env_left.append(np.identity((lam[i].shape[0])))
    Env_right.append(np.dot(np.dot(np.diag(lam[i+1]),np.identity((lam[i+1].shape[0]))),np.diag(lam[i+1])))

print("Truncation: chi_max = "+repr(chi_max))
print("Energy of MPS with truncation = "+repr(TEBD.Calc_Energy(Env_left,Env_right,Tn,lam,Jz,Jxy,hx,D)))

Truncation: chi_max = 10
Energy of MPS with truncation = -1.4282534935314155
